# StepWise Controllable Agent

This notebook introduces you to our advanced lower-level agent API designed to provide enhanced functionality beyond just executing user queries.

It will help you in creating tasks, navigating through various steps.


## Installation

In [ ]:
!pip install llama-index
!pip install llama-index-llms-openai

In [ ]:
from llama_index.llms.openai import OpenAI
from llama_index.core.llms import ChatMessage
from llama_index.core.tools import BaseTool, FunctionTool
from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.agent import AgentRunner, ReActAgent

## Setup API Key

In [ ]:
import os
os.environ['OPENAI_API_KEY'] = 'YOUR OPENAI API KEY'

## Simple Calculator Tools

### Define Tools

In [ ]:
def multiply(a: int, b: int) -> int:
    """Multiply two integers and returns the result integer"""
    return a * b

def add(a: int, b: int) -> int:
    """Add two integers and returns the result integer"""
    return a + b

def subtract(a: int, b: int) -> int:
    """Subtract two integers and returns the result integer"""
    return a - b

In [ ]:
multiply_tool = FunctionTool.from_defaults(fn=multiply)
add_tool = FunctionTool.from_defaults(fn=add)
subtract_tool = FunctionTool.from_defaults(fn=subtract)

### Setup Agent

In [ ]:
llm = OpenAI(model="gpt-4")
agent_worker = FunctionCallingAgentWorker.from_tools(
    [multiply_tool, add_tool, subtract_tool],
    llm=llm,
    verbose=True,
    allow_parallel_tool_calls=False,
)
agent = AgentRunner(agent_worker)

In [ ]:
response = agent.chat("What is 20+(2*4)-8?")

Added user message to memory: What is 20+(2*4)-8?
=== Calling Function ===
Calling function: multiply with args: {"a": 2, "b": 4}
=== Function Output ===
8
=== Calling Function ===
Calling function: add with args: {"a": 20, "b": 8}
=== Function Output ===
28
=== Calling Function ===
Calling function: subtract with args: {"a": 28, "b": 8}
=== Function Output ===
20
=== LLM Response ===
The result of the operation 20+(2*4)-8 is 20.


In [ ]:
print(response)

assistant: The result of the operation 20+(2*4)-8 is 20.


### StepWise Execution

In [ ]:
# start task
task = agent.create_task("What is 20+(2*4)-8?")

In [ ]:
step_output = agent.run_step(task.task_id)

Added user message to memory: What is 20+(2*4)-8?
=== Calling Function ===
Calling function: multiply with args: {"a": 2, "b": 4}
=== Function Output ===
8


In [ ]:
step_output

TaskStepOutput(output=AgentChatResponse(response='assistant: None', sources=[ToolOutput(content='8', tool_name='multiply', raw_input={'args': (), 'kwargs': {'a': 2, 'b': 4}}, raw_output=8, is_error=False)], source_nodes=[], is_dummy_stream=False), task_step=TaskStep(task_id='ade5dc7d-0286-46d8-965f-8761c8053aae', step_id='fb2a4ffe-5f7c-4d80-a803-a16c49772b0b', input='What is 20+(2*4)-8?', step_state={}, next_steps={}, prev_steps={}, is_ready=True), next_steps=[TaskStep(task_id='ade5dc7d-0286-46d8-965f-8761c8053aae', step_id='13daf43a-b061-4cd6-87fe-47d1da610b75', input=None, step_state={}, next_steps={}, prev_steps={}, is_ready=True)], is_last=False)

In [ ]:
# Check if its last step
print(step_output.is_last)

False


In [ ]:
step_output = agent.run_step(task.task_id)

=== Calling Function ===
Calling function: add with args: {"a": 20, "b": 8}
=== Function Output ===
28


In [ ]:
# Check if its last step
print(step_output.is_last)

False


In [ ]:
step_output = agent.run_step(task.task_id)

=== Calling Function ===
Calling function: subtract with args: {"a": 28, "b": 8}
=== Function Output ===
20


In [ ]:
# Check if its last step
print(step_output.is_last)

False


In [ ]:
step_output = agent.run_step(task.task_id)

=== LLM Response ===
The result of the operation 20+(2*4)-8 is 20.


In [ ]:
# Check if its last step
print(step_output.is_last)

True


In [ ]:
# now that the step execution is done, we can finalize response
response = agent.finalize_response(task.task_id)
print(str(response))

assistant: The result of the operation 20+(2*4)-8 is 20.


## RAG QueryEngine Tools

In [ ]:
from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage,
)

from llama_index.core.tools import QueryEngineTool, ToolMetadata

### Download Data

In [ ]:
!mkdir -p 'data/10q/'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10q/uber_10q_march_2022.pdf' -O 'data/10q/uber_10q_march_2022.pdf'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10q/uber_10q_june_2022.pdf' -O 'data/10q/uber_10q_june_2022.pdf'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10q/uber_10q_sept_2022.pdf' -O 'data/10q/uber_10q_sept_2022.pdf'

--2024-04-13 00:31:47--  https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10q/uber_10q_march_2022.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1260185 (1.2M) [application/octet-stream]
Saving to: ‘data/10q/uber_10q_march_2022.pdf’

data/10q/uber_10q_m 100%[===================>]   1.20M  --.-KB/s    in 0.05s   

2024-04-13 00:31:48 (22.0 MB/s) - ‘data/10q/uber_10q_march_2022.pdf’ saved [1260185/1260185]

--2024-04-13 00:31:48--  https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10q/uber_10q_june_2022.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercont

### Load Data

In [ ]:
march_2022 = SimpleDirectoryReader(
    input_files=["./data/10q/uber_10q_march_2022.pdf"]
).load_data()
june_2022 = SimpleDirectoryReader(
    input_files=["./data/10q/uber_10q_june_2022.pdf"]
).load_data()
sept_2022 = SimpleDirectoryReader(
    input_files=["./data/10q/uber_10q_sept_2022.pdf"]
).load_data()

### Define Tools

In [ ]:
import os


def get_tool(name, full_name, documents=None):
    if not os.path.exists(f"./data/{name}"):
        # build vector index
        vector_index = VectorStoreIndex.from_documents(documents)
        vector_index.storage_context.persist(persist_dir=f"./data/{name}")
    else:
        vector_index = load_index_from_storage(
            StorageContext.from_defaults(persist_dir=f"./data/{name}"),
        )
    query_engine = vector_index.as_query_engine(similarity_top_k=3, llm=llm)
    query_engine_tool = QueryEngineTool(
        query_engine=query_engine,
        metadata=ToolMetadata(
            name=name,
            description=(
                "Provides information about Uber quarterly financials ending"
                f" {full_name}"
            ),
        ),
    )
    return query_engine_tool

In [ ]:
march_tool = get_tool("march_2022", "March 2022", documents=march_2022)
june_tool = get_tool("june_2022", "June 2022", documents=june_2022)
sept_tool = get_tool("sept_2022", "September 2022", documents=sept_2022)

In [ ]:
query_engine_tools = [march_tool, june_tool, sept_tool]

### Create Agent

In [ ]:
llm = OpenAI('gpt-4')

agent_worker = FunctionCallingAgentWorker.from_tools(
    query_engine_tools,
    llm=llm,
    verbose=True,
    allow_parallel_tool_calls=False,
)
agent = AgentRunner(agent_worker)

#### Direct Execution

In [ ]:
response = agent.chat("Analyze the changes in R&D expenditures and revenue.")

Added user message to memory: Analyze the changes in R&D expenditures and revenue.
=== LLM Response ===
To analyze the changes in R&D expenditures and revenue, we need to fetch the data for the respective quarters. Let's start with the first quarter ending in March 2022.
=== Calling Function ===
Calling function: march_2022 with args: {"input": "R&D expenditures and revenue"}
=== Function Output ===
Research and development expenditures for Uber increased by $72 million, or 14%, from $515 million in 2021 to $587 million in 2022. This increase was primarily due to a $63 million increase in stock-based compensation expense. The percentage of revenue attributed to research and development decreased from 18% in 2021 to 9% in 2022. 

On the other hand, Uber's revenue increased by $4.0 billion, or 136%, from $2,903 million in 2021 to $6,854 million in 2022. This increase was primarily driven by a 35% increase in Gross Bookings, a $1.5 billion increase in Freight revenue due to the acquisitio

In [ ]:
print(response)

assistant: In the third quarter ending in September 2022, Uber's R&D expenses were $760 million, which was a 54% increase from the same period in 2021. For the nine months ending September 30, 2022, R&D expenses were $2,051 million, a 37% increase from the same period in 2021. The R&D expenses as a percentage of revenue were 9% for both the three and nine months periods ending September 30, 2022. The increase in R&D expenses was primarily due to increases in stock-based compensation and employee headcount costs.

In summary, Uber's R&D expenditures have been increasing throughout 2022, primarily due to increases in stock-based compensation and employee headcount costs. However, the percentage of revenue attributed to R&D has remained consistent at 9%. This suggests that while Uber is investing more in R&D, its revenue is growing at a pace that allows the company to maintain its investment in R&D as a consistent percentage of its revenue.


### Step Wise Execution

In [ ]:
agent_worker = FunctionCallingAgentWorker.from_tools(
    query_engine_tools,
    llm=llm,
    verbose=True,
    allow_parallel_tool_calls=False,
)
agent = AgentRunner(agent_worker)

In [ ]:
# start task
task = agent.create_task("Analyze the changes in R&D expenditures and revenue.")

In [ ]:
step_output = agent.run_step(task.task_id)

Added user message to memory: Analyze the changes in R&D expenditures and revenue.
=== LLM Response ===
To analyze the changes in R&D expenditures and revenue, we need to pull the data for these metrics from the quarterly financials. Let's start with the first quarter of 2022.

{ "input": "R&D expenditures and revenue" }
=== Calling Function ===
Calling function: march_2022 with args: {"input": "R&D expenditures and revenue"}
=== Function Output ===
The research and development expenditures for Uber increased from $515 million in the first quarter of 2021 to $587 million in the first quarter of 2022, marking a 14% change. This increase was primarily due to a $63 million increase in stock-based compensation expense. In terms of revenue, Uber saw a significant increase from $2.9 billion in the first quarter of 2021 to $6.85 billion in the first quarter of 2022, a 136% change. This increase in revenue was primarily due to an increase in Gross Bookings, a $1.5 billion increase in Freight r

In [ ]:
# Check if its last step
print(step_output.is_last)

False


In [ ]:
step_output = agent.run_step(task.task_id)

=== LLM Response ===
In the first quarter of 2022, Uber's R&D expenditures increased by 14% from the same period in 2021, rising from $515 million to $587 million. This increase was mainly due to a $63 million increase in stock-based compensation expense. 

In terms of revenue, there was a significant increase of 136% from $2.9 billion in Q1 2021 to $6.85 billion in Q1 2022. This was primarily due to an increase in Gross Bookings, a $1.5 billion increase in Freight revenue, and a $304 million increase in Delivery revenue.

Now, let's look at the data for the second quarter of 2022.

{ "input": "R&D expenditures and revenue" }
=== Calling Function ===
Calling function: june_2022 with args: {"input": "R&D expenditures and revenue"}
=== Function Output ===
In the three months ended June 30, 2022, Uber's research and development expenses increased by $216 million, or 44%, compared to the same period in 2021. This was primarily due to a $128 million increase in stock-based compensation and 

In [ ]:
# Check if its last step
print(step_output.is_last)

False


In [ ]:
step_output = agent.run_step(task.task_id)

=== LLM Response ===
In the second quarter of 2022, Uber's R&D expenses increased by 44% compared to the same period in 2021, with an increase of $216 million. This was primarily due to a $128 million increase in stock-based compensation and a $90 million increase in employee headcount costs. 

For the first half of 2022, R&D expenses increased by 29% compared to the same period in 2021, with an increase of $288 million. This was mainly due to a $191 million increase in stock-based compensation and a $145 million increase in employee headcount costs. 

The percentage of revenue attributed to research and development was 9% for both the second quarter and the first half of 2022.

Now, let's look at the data for the third quarter of 2022.

{ "input": "R&D expenditures and revenue" }
=== Calling Function ===
Calling function: sept_2022 with args: {"input": "R&D expenditures and revenue"}
=== Function Output ===
In 2022, Uber's research and development expenses increased significantly. For

In [ ]:
# Check if its last step
print(step_output.is_last)

False


In [ ]:
step_output = agent.run_step(task.task_id)

=== LLM Response ===
In the third quarter of 2022, Uber's R&D expenses increased by 54% compared to the same period in 2021, with an increase of $267 million. This was primarily due to a $140 million increase in stock-based compensation and a $129 million increase in employee headcount costs. 

For the first nine months of 2022, R&D expenses increased by 37% compared to the same period in 2021, with an increase of $555 million. This was mainly due to a $331 million increase in stock-based compensation and a $274 million increase in employee headcount costs. 

The percentage of revenue attributed to research and development was 9% for both the third quarter and the first nine months of 2022.

In summary, Uber has been significantly increasing its investment in R&D throughout 2022, with a consistent 9% of revenue being allocated to this area. This increased investment is reflected in the rising costs of stock-based compensation and employee headcount.


In [ ]:
# Check if its last step
print(step_output.is_last)

True


In [ ]:
response = agent.finalize_response(task.task_id)

In [ ]:
print(response)

assistant: In the third quarter of 2022, Uber's R&D expenses increased by 54% compared to the same period in 2021, with an increase of $267 million. This was primarily due to a $140 million increase in stock-based compensation and a $129 million increase in employee headcount costs. 

For the first nine months of 2022, R&D expenses increased by 37% compared to the same period in 2021, with an increase of $555 million. This was mainly due to a $331 million increase in stock-based compensation and a $274 million increase in employee headcount costs. 

The percentage of revenue attributed to research and development was 9% for both the third quarter and the first nine months of 2022.

In summary, Uber has been significantly increasing its investment in R&D throughout 2022, with a consistent 9% of revenue being allocated to this area. This increased investment is reflected in the rising costs of stock-based compensation and employee headcount.


### Step Wise Execution With Human Feedback

In [ ]:
agent_worker = FunctionCallingAgentWorker.from_tools(
    query_engine_tools,
    llm=llm,
    verbose=True,
    allow_parallel_tool_calls=False,
)
agent = AgentRunner(agent_worker)

In [ ]:
# start task
task = agent.create_task("Analyze the changes in R&D expenditures and revenue.")

In [ ]:
step_output = agent.run_step(task.task_id)

Added user message to memory: Analyze the changes in R&D expenditures and revenue.
=== LLM Response ===
To analyze the changes in R&D expenditures and revenue, we need to pull the data for these metrics from the financial reports for the relevant quarters. Let's start with the first quarter of 2022.
=== Calling Function ===
Calling function: march_2022 with args: {"input": "R&D expenditures and revenue"}
=== Function Output ===
The research and development expenditures for Uber increased from $515 million in the first quarter of 2021 to $587 million in the same period in 2022, marking a 14% increase. This increase was primarily due to a $63 million increase in stock-based compensation expense. In terms of revenue, Uber saw a significant increase from $2.9 billion in the first quarter of 2021 to $6.85 billion in the first quarter of 2022, a 136% increase. This increase in revenue was primarily due to a 35% increase in Gross Bookings, a $1.5 billion increase in Freight revenue, and a $30

In [ ]:
# Check if its last step
print(step_output.is_last)

False


In [ ]:
step_output = agent.run_step(task.task_id, input='What about September?')

Added user message to memory: What about September?
=== Calling Function ===
Calling function: sept_2022 with args: {"input": "R&D expenditures and revenue"}
=== Function Output ===
In the three months ending September 30, 2022, Uber's research and development expenses were $760 million, which was a 54% increase from the same period in 2021. For the nine months ending September 30, 2022, the research and development expenses were $2,051 million, a 37% increase from the same period in 2021. The percentage of revenue spent on research and development was 9% for both the three and nine month periods in 2022. The increase in expenses was primarily due to a rise in stock-based compensation and employee headcount costs.
